# Lab 6

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/geog-414/blob/master/book/labs/lab_06.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

In [ ]:
import ee 
import geemap
ee.Initialize()

## Question 1

Create a map to visualize [NOAA GFS Temperature Data](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25) and add a color bar and [NOAA logo](https://i.imgur.com/spILFEi.png) to the map. 

In [ ]:
Map= geemap.Map()

collection = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2018-12-22', '2018-12-23')
    .limit(24)
    .select('temperature_2m_above_ground')
)

image= collection.median()
vis_params = {
    'min': -40.0,
    'max': 35.0,
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.addLayer(image, vis_params, 'Temperature')
Map

In [ ]:
Map.add_colorbar(vis_params, label=" Temperature Abover Ground", layer_name="Temperature")

In [ ]:
text= 'made by Zhanchao Yang'
Map.add_text(text, position='bottomright')

In [ ]:
logo = 'NOAA.png'
Map.add_image(logo, position='bottomleft')

![](https://i.imgur.com/bBoRswR.png)

## Question 2

**Linked Maps:** Create a 2*2 linked map to visualize the Landsat imagery (`ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`) with different band combinations.

In [ ]:
import geemap
import ee

In [ ]:
ee.Initialize()

In [ ]:

image = landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)

vis_params = [
    {'bands': ['B3', 'B2', 'B1'], 'gamma': 1.4},
    {'bands': ['B4', 'B5', 'B3'], 'gamma': 1.4},
    {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4},
    {'bands': ['B5', 'B4', 'B3'], 'gamma': 1.4},
]

labels = [
    'Natural Color (B3/B2/B1)',
    'Land/Water (B4/B5/B3)',
    'Color Infrared (B8/B4/B3)',
    'Vegetation (B5/B4/B3)',
]

geemap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    center=[38.4151, 21.2712],
    zoom=12,
    ee_objects=[image],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)
     

![](https://i.imgur.com/Xp15uOo.png)

## Question 3

**Timeseries Inspector:** Create a map with timeseries inspector to visualize [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL) from 2010 to 2022. Add your name and [USDA logo](https://i.imgur.com/tzH2dNr.png) to the map.

In [ ]:
Map= geemap.Map(center=[40,-100], zoom=4)
collection= ee.ImageCollection("USDA/NASS/CDL").select('cropland')
vis_params= {'bands': ['cropland']}
years= collection.aggregate_array('system:index').getInfo()

In [ ]:
Map.ts_inspector(
    left_ts=collection, 
    right_ts=collection, 
    left_names=years, 
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)
Map

In [ ]:
picture= 'usda.png'
Map.add_image(picture, position='bottomleft')

text= 'Made by Zhanchao Yang'
Map.add_text(text, position='bottomright')

![](https://i.imgur.com/gW1XPhL.png)

## Question 4

**Time slider:** Create a map with the time slider to visualize [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) for Knoxville, TN.

In [ ]:
import geemap
import ee 
ee.Initialize()


In [ ]:
fc= ee.FeatureCollection('TIGER/2018/Counties')
subset= fc.filter(ee.Filter.eq('NAME', 'Knox'))
knox= subset.filter(ee.Filter.eq('STATEFP', '47'))

In [ ]:
collection= (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2021-01-01', '2021-07-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .filterBounds(knox)
)
collection

In [ ]:
Map= geemap.Map(center=[40,-100], zoom=4)
collection= (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2021-01-01', '2021-01-20')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .filterBounds(knox)
    .select(['B3', 'B4', 'B8'])
)
vis_params= {
    'bands': ['B8', 'B4', 'B3'], 
    'min': 0, 
    'max': 2000
}

Map.add_time_slider(collection, vis_params, time_interval=2)
Map.centerObject(knox, zoom=10)
Map



In [ ]:
text= 'Made by Zhanchao Yang'
Map.add_text(text, position='bottomright')

![](https://i.imgur.com/rJe2rvP.png)

## Question 5

**Split-panel Map:** Use the following datasets to create a split-panel map for visualizing the ESA land cover data in the US. Add the ESA land cover legend to the map (Hint: the built-in legend for ESA land cover is `ESA_WorldCover`).

- [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States): `ee.FeatureCollection("TIGER/2018/States")`
- [ESA WorldCover 10m](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100): `ee.ImageCollection("ESA/WorldCover/v100")`
- Landsat: `LANDSAT/LE7_TOA_5YEAR/1999_2003`


Currently, the split-map control of ipyleaflet plotting backend has a bug ([source](https://github.com/jupyter-widgets/ipyleaflet/issues/1066)). Use the folium plotting backend instead.

In [1]:
import geemap
import ee 
ee.Initialize()

In [2]:
states= ee.FeatureCollection('TIGER/2018/States')

landcover_collection= (ee.ImageCollection('ESA/WorldCover/v100')
                       .select('Map').first())
subset1= landcover_collection.clip(states)

Landsat_collection= (ee.ImageCollection('LANDSAT/LE7_TOA_5YEAR')
                     .filterDate('1999-01-01', '1999-12-31')
)
Lansat_image= Landsat_collection.median()
subset2= Lansat_image.clip(states)

In [7]:
Map= geemap.Map(center=[40,-100], zoom=4)
vis_landsat={'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
right_layer= geemap.ee_tile_layer(subset1, vis_params={}, name='Land Cover')
left_layer= geemap.ee_tile_layer(subset2, vis_landsat, name='Landsat')
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
Map.add_legend(builtin_legend='ESA_WorldCover')

In [9]:
text= 'Made by Zhanchao Yang'
Map.add_text(text, position='bottomright')

![](https://i.imgur.com/y0GW6k8.png)